# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import geoviews.operation as gvo
# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,coronel dorrego,-38.7187,-61.2873,23.19,73,23,5.20,AR,Wed Jan 31 19:03:19 2024
1,1,west island,-12.1568,96.8225,29.99,70,40,3.09,CC,Wed Jan 31 19:03:19 2024
2,2,margaret river,-33.9500,115.0667,28.47,50,0,2.52,AU,Wed Jan 31 19:03:19 2024
3,3,yuzhno-kurilsk,44.0306,145.8556,-2.01,71,36,14.14,RU,Wed Jan 31 19:03:19 2024
4,4,klichaw,53.4928,29.3352,1.38,98,100,5.03,BY,Wed Jan 31 19:03:20 2024


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    color= 'City',
    size= 'Humidity',
    hover_cols = ["City", "Hotel Name", "Country"]
)
#display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()
city_data_df.count()

City_ID       588
City          588
Lat           588
Lng           588
Max Temp      588
Humidity      588
Cloudiness    588
Wind Speed    588
Country       583
Date          588
dtype: int64

### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''


# Display sample data
hotel_df 

,City,Country,Lat,Lng,Humidity,Hotel Name
7,santa rosa de rio primero,AR,-31.1523,-63.4019,45,
245,mar del plata,AR,-38.0023,-57.5575,50,
255,mermoz boabab,SN,14.7065,-17.4758,73,
337,punta del este,UY,-34.9667,-54.9500,84,
420,santo antonio,BR,-6.3106,-35.4789,80,
457,xai-xai,MZ,-25.0519,33.6442,92,
487,san luis,AR,-33.2950,-66.3356,42,
509,corzuela,AR,-26.9537,-60.9693,42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'limit': 20,  
    'apiKey': geoapify_key
}
def get_hotel_info(city, country, latitude, longitude):
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Iterate over rows in ideal_weather_df and populate hotel_df
for index, row in ideal_weather_df.iterrows():
    # Print a message to follow up the hotel search
    print("Starting hotel search")
    get_hotel_info(row['City'], row['Country'], row['Lat'], row['Lng'])


Starting hotel search
santa rosa de rio primero - nearest hotel: Hotel presidente
Starting hotel search
mar del plata - nearest hotel: Gran hotel Mónaco
Starting hotel search
mermoz boabab - nearest hotel: La maison blanche
Starting hotel search
punta del este - nearest hotel: Hotel Aquarium
Starting hotel search
santo antonio - nearest hotel: Pousada Onça Puma
Starting hotel search
xai-xai - nearest hotel: No hotel found
Starting hotel search
san luis - nearest hotel: Hotel Dos Venados
Starting hotel search
corzuela - nearest hotel: Alamos


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
hotel_points = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    color= 'City',
    size= 'Humidity',
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
hotel_points

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)